# Finding validation accuracy for temporal models

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf
from tensorflow.keras import Model
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import glob
import cv2
import TransformCode as tc
from io import BytesIO
from PIL import Image
from numpy import expand_dims
#from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import *
import warnings
import logging
from IPython.display import clear_output
from collections import Counter
import pickle
import json
import sys
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/augmentations')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/hyperparameters')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/visualisations')
import VisualisationTools as plotting
import hyper_utils as hp
from efficientnet.tfkeras import EfficientNetB0

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(100)
plot_losses = plotting.PlotLearning()
os.chdir('/home/jupyter/DeepFake-2019-20')

print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.3.0


### Create the data generators
The parameters are specified manually as they are consistent for each config.

In [3]:
import os
import glob
import VideoFrameGenerator

# use sub directories names as classes
classes = [i.split(os.path.sep)[-1] for i in glob.glob('../restructured-all-faces/home/jupyter/restructured_data/validation/*')]
classes.sort() 
print(classes)

SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 20
BS = 4

glob_pattern_test = '../restructured-all-faces/home/jupyter/restructured_data/validation/{classname}/*'

data_aug_test = tf.keras.preprocessing.image.ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
test_data = VideoFrameGenerator.VideoFrameGenerator(
        classes=classes, 
        glob_pattern=glob_pattern_test,
        nb_frames=NBFRAME,
        shuffle=True,
        batch_size=BS,
        target_shape=SIZE,
        nb_channel=CHANNELS,
        transformation=data_aug_test)

['authentic', 'fake']
Total data: 2 classes for 700 files for train


### Importing the saved models

In [ ]:
af_dir = '../all_faces_bucket/'
configname = 'config1011'
from tensorflow.keras.utils import CustomObjectScope

with CustomObjectScope({'TransformerBlock': tc.TransformerBlock, 'MultiHeadSelfAttention' : tc.MultiHeadSelfAttention}):
    model = tf.keras.models.load_model(af_dir + 'trained_models/saved_models/' + configname + '.h5')

### Running the evaluation
Instead of predicting on the validation set, the model is evaluated directly as this model does not require frame predictions to be transferred to video predictions.

In [ ]:
score, acc = model.evaluate(test_data,
                           # class_weight=class_weights,
                          #  initial_epoch=0,
                            verbose=1,
                            max_queue_size=100,                # maximum size for the generator queue
                            workers=16,                        # maximum number of processes to spin up when using process-based threading
                            use_multiprocessing=False)